In [2]:
import os
from glob import glob
import pandas as pd

import wmfdata as wmf
from wmfdata import charting, mariadb, hive
from wmfdata.utils import pct_str, pd_display_all
from urllib.parse import unquote

You are using wmfdata 0.1.0 (latest).

You can find the source for `wmfdata` at https://github.com/neilpquinn/wmfdata


In [16]:
filename = '../../data/raw/articles/2019/Tiger 2.0 - article page stats.xlsx'
cols2skip = [0, 2,3,4]
cols = [i for i in range(10) if i not in cols2skip]
    
dfs = pd.read_excel(filename, sheet_name=None, usecols=cols) #skiprows=1


In [17]:
df2 = pd.concat([df.assign(name=n) for n,df in dfs.items()], sort=True)

In [18]:
df2

,name,بچوں تک رسائی,पृथुः,माजुरो,हवामानदर्शक नकाशा,আনোৱাৰুদ্দিন চৌধুৰী,দামোহ লোকসভা কেন্দ্র,ਫਰੈਡਰਿਕ ਸੇਂਜਰ,બિનોદ બિહારી ચૌધરી,ନିକିତା କୃସ୍ଚେଭ,புருசோத்தம் லட்சுமண் தேசுபாண்டே,హలో గురు ప్రేమకోసమే,ಇಳಂಗೋ ಅಡಿಗಳ್,ಕಲ್ಯಾಣ ಸ್ವಾಮಿ,മിങ് ശവകുടീരങ്ങൾ,ᱡᱮᱵᱷᱤᱭᱟᱨ ᱵᱮᱴᱴᱮᱞ
0,Assamese,NaN,NaN,NaN,NaN,কাৱঁড় যাত্ৰা,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Assamese,NaN,NaN,NaN,NaN,ভটিমা,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Assamese,NaN,NaN,NaN,NaN,বগা বাবাৰ মাজাৰ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Assamese,NaN,NaN,NaN,NaN,পোৱামক্কা,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Assamese,NaN,NaN,NaN,NaN,ডেকা চাং,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1390,Urdu,مہابھارت,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1391,Urdu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1392,Urdu,مومنہ درید,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1393,Urdu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
#df = pd.read_csv("../../data/raw/articles/2019/Tiger 2.0 - article page stats.xlsx")



import pandas as pd

sheets_dict = pd.read_excel('../../data/raw/articles/2019/Tiger 2.0 - article page stats.xlsx', sheet_name=None)

full_table = pd.DataFrame()
for name, sheet in sheets_dict.items():
    sheet['sheet'] = name
    sheet = sheet.rename(columns=lambda x: x.split('\n')[-1])
    full_table = full_table.append(sheet)

full_table.reset_index(inplace=True, drop=True)

full_table.head(2)

/home/iflorez/venv/lib/python3.5/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


AttributeError: 'float' object has no attribute 'split'

In [83]:
#add wiki_db column for querying
df['wiki_db'] = df['Language']+'wiki'

In [11]:
contest_titles_denormalized = tuple(list(df['Articles']))

In [33]:
article_vars = dict(
    contest_titles_denormalized = contest_titles_denormalized,
    wiki_dbs = wiki_dbs
)

In [35]:
#get qids 
qid_r = wmf.mariadb.run("""
SELECT
  ips_site_page AS article,
  ips_item_id AS QID
FROM  wb_items_per_site  
WHERE ips_site_id IN {wiki_dbs} 
      AND ips_site_page IN {contest_titles_denormalized}
""".format(**article_vars), "wikidatawiki")

In [159]:
#get clean list

#merge in the clean list into the df
df_w_ids = pd.merge(df, qid_r[['article', 'QID']], how="left", left_on=['Articles'], right_on=['article']).drop('article', axis=1)

#CLEAN DF - ready to use, drop na, drop all dupes
df_w_ids_no_nulls = df_w_ids[df_w_ids['QID'].notna()]
df_w_ids_clean = df_w_ids_no_nulls.drop_duplicates(subset=['Language', 'Articles', 'wiki_db', 'QID'], keep=False)

In [145]:
#MISSING DF - to add data
articles_w_QID_missing = df_w_ids[df_w_ids['QID'].isnull()]

In [179]:
#identify dupes 
#articles_w_QID_duplicated = df_w_ids_no_nulls[df_w_ids_no_nulls.duplicated(subset=['Language', 'Articles', 'wiki_db', 'QID'], keep=False)]

#keep only first instance of dupes of full row duplicates
articles_w_QID_duplicated = df_w_ids_no_nulls[df_w_ids_no_nulls.duplicated(subset=['Language', 'Articles', 'wiki_db', 'QID'], keep='first')]

In [90]:
#MISSING DF - to add data
articles_w_QID_missing

#DUPES DF - to clean 
articles_w_QID_duplicated

#CLEAN DF - ready to use
df_w_ids_clean

In [186]:
df_updated = articles_w_QID_duplicated.append(df_w_ids_clean)

In [190]:
articles_w_QID_missing.to_csv("../../data/raw/articles/2019/articles_QID_missing.csv", sep=',', index=False, encoding='utf-8')

In [191]:
df_updated.to_csv("../../data/raw/articles/2019/contest_titles_n_updated.csv", sep=',', index=False, encoding='utf-8')